In [19]:
import os
import numpy as np
import mne
from pathlib import Path
from mne.preprocessing import ICA
from mne.preprocessing import (ICA, corrmap, create_ecg_epochs,
                               create_eog_epochs)
import matplotlib.pyplot as pl

# the following import is required for matplotlib < 3.2:
from mpl_toolkits.mplot3d import Axes3D  # noqa

In [ ]:
# Charger cVEMPs
sample_data_folder = 'D:/Stage_M2/'
sample_data_raw_file = sample_data_folder + 'YBassis.bdf'
raw = mne.io.read_raw_bdf(sample_data_raw_file,preload=True)

In [ ]:
#Charger oVEMPs 
sample_data_folder = 'D:/Stage_M2_Paul/'
sample_data_raw_file = sample_data_folder + 'JKassis.bdf'
raw = mne.io.read_raw_bdf(sample_data_raw_file,preload=True)

In [ ]:
# change les noms des électrodes
channel_renaming_dict = {'A1':'Fp1','A2':'AF7','A3':'AF3','A4':'F1','A5':'F3','A6':'F5','A7':'F7','A8':'FT7','A9':'FC5',
                        'A10':'FC3','A11':'FC1','A12':'C1','A13':'C3','A14':'C5','A15':'T7','A16':'TP7','A17':'CP5',
                        'A18':'CP3','A19':'CP1','A20':'P1','A21':'P3','A22':'P5','A23':'P7','A24':'P9','A25':'PO7',
                         'A26':'PO3','A27':'O1','A28':'Iz','A29':'Oz','A30':'POz','A31':'Pz','A32':'CPz','B1':'Fpz',
                         'B2':'Fp2','B3':'AF8','B4':'AF4','B5':'AFz','B6':'Fz','B7':'F2','B8':'F4','B9':'F6',
                        'B10':'F8','B11':'FT8','B12':'FC6','B13':'FC4','B14':'FC2','B15':'FCz','B16':'Cz',
                         'B17':'C2','B18':'C4','B19':'C6','B20':'T8','B21':'TP8','B22':'CP6','B23':'CP4','B24':'CP2',
                         'B25':'P2','B26':'P4','B27':'P6','B28':'P8','B29':'P10','B30':'PO8','B31':'PO4','B32':'O2',}
raw.rename_channels(channel_renaming_dict)

# cVEMPs

In [ ]:
# Marque les électrodes bruitées
raw.info['bads'] = []
# Marque les électrodes de références
raw.set_channel_types(mapping={'IOd': 'eog'})
raw.set_channel_types(mapping={'IOg': 'eog'})
raw.set_channel_types(mapping={'SOd': 'eog'})
raw.set_channel_types(mapping={'SCMd': 'eeg'})
raw.set_channel_types(mapping={'SCMg': 'emg'})
raw.set_channel_types(mapping={'Sternum': 'eeg'})
raw.set_channel_types(mapping={'Eard': 'eeg'})
raw.set_channel_types(mapping={'Earg': 'bio'})
picks_eeg = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False, ecg=False,exclude='bads')
picks_eeg_ecg = mne.pick_types(raw.info, meg=False, ecg=True, eeg=False, eog=False, stim=False,exclude='bads')
picks_eog = mne.pick_types(raw.info, meg=False, eeg=False, eog=True, stim=False, ecg=False,exclude='bads')


# Mettre Sternum en référence pour SCMd
mne.set_eeg_reference(raw, ref_channels=['Sternum'])

# oVEMPs

In [ ]:
# Marque les électrodes bruitées
raw.info['bads'] = []
# Marque les électrodes de références
raw.set_channel_types(mapping={'IOd': 'eog'})
raw.set_channel_types(mapping={'IOg': 'eeg'})
raw.set_channel_types(mapping={'SOd': 'eog'})
raw.set_channel_types(mapping={'SCMd': 'eeg'})
raw.set_channel_types(mapping={'SCMg': 'emg'})
raw.set_channel_types(mapping={'Sternum': 'eeg'})
raw.set_channel_types(mapping={'Eard': 'eeg'})
raw.set_channel_types(mapping={'Earg': 'bio'})
picks_eeg = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False, ecg=False,exclude='bads')
picks_eeg_ecg = mne.pick_types(raw.info, meg=False, ecg=True, eeg=False, eog=False, stim=False,exclude='bads')
picks_eog = mne.pick_types(raw.info, meg=False, eeg=False, eog=True, stim=False, ecg=False,exclude='bads')


# Mettre Sternum en référence pour Eard
mne.set_eeg_reference(raw, ref_channels=['Eard'])

In [ ]:
# Visualiser données brutes
raw.plot()

In [ ]:
#Apply x(-1) to Status channel to correctly detect triggers afterwards with find_events()
raw.apply_function(np.negative,picks=['Status'])
# filtrer le signal (passe-haut puis passe-bas)
raw_highpass = raw.filter(l_freq=1, h_freq=None)
raw_bandpass = raw_highpass.filter(l_freq=None, h_freq=200)

# Events

In [ ]:
#find events
events = mne.find_events(raw_bandpass,stim_channel='Status',uint_cast=True)
print(events.shape)
#annote raw data with triggers
annot_from_events = mne.annotations_from_events(
    events=events, sfreq=raw.info['sfreq'],
    orig_time=raw.info['meas_date'])
raw.set_annotations(annot_from_events)
#annote raw data with triggers
annot_from_events = mne.annotations_from_events(
    events=events, sfreq=raw.info['sfreq'],
    orig_time=raw.info['meas_date'])
raw.set_annotations(annot_from_events)
# exclure les pauses comme "bad signal"
break_annots = mne.preprocessing.annotate_break(
    raw=raw,
    min_break_duration=4,  # consider segments of at least 20 s duration
    t_start_after_previous=0.4,  # start annotation 5 s after end of previous one
    t_stop_before_next=0.03  # stop annotation 2 s before beginning of next one
)
raw.set_annotations(raw.annotations + break_annots)  # add to existing
#rename triggers
event_dict = {'500Hz105dB': 2,
              '2500Hz105dB': 5, 
              '500Hz60dB': 3,
              'mask500Hz90dB': 9 }

In [ ]:
raw.plot()

# cVEMPS

In [ ]:
# segmente le signal en epochs (cVEMPs intervalle de temps allant de -0.05s avant le stim à +0,300s après le stim)
#baseline
baseline= (-0.05,0)
epochs = mne.Epochs(raw_bandpass, events, event_id=event_dict , tmin=-0.05, tmax=0.3, picks=picks_eeg,baseline=baseline,
                    preload=True)
# Visualise les epochs (retirer les epochs bruitées)
#fig = epochs.plot(events=events)

# oVEMPs

In [ ]:
# segmente le signal en epochs (oVEMPs intervalle de temps allant de -0.05s avant le stim à +0,300s après le stim)
#baseline
baseline= (-0.05,0)
epochs = mne.Epochs(raw_bandpass, events, event_id=event_dict , tmin=-0.05, tmax=0.3, picks=picks_eeg,baseline=baseline,
                    preload=True)
# Visualise les epochs (retirer les epochs bruitées)
#fig = epochs.plot(events=events)

In [ ]:
# sauvegarder le fichier epochs après le retrait des badepochs
epochs.save('D:/Stage_M2_Paul/PK/PKcVEMPs.fif')

# oVEMPs

In [ ]:
# sauvegarder le fichier epochs après le retrait des badepochs
epochs.save('D:/Stage_M2_Paul/PK/PKoVEMPs.fif')

# cVEMPs

In [ ]:
# Extraire les moyennes de l'électrode SCMd et du Sternum
emg_epochs = epochs.copy().pick_channels(['SCMd', 'Sternum'])
emg_evoked = emg_epochs.average()

# Visualiser le signal de SCMd avec la référence Sternum ou right Earlobe
emg_evoked.plot()

In [ ]:
# Extraire les moyennes de l'électrode IOg et du Eard
emg_epochs = epochs.copy().pick_channels(['IOg', 'Eard'])
emg_evoked = emg_epochs.average()

# Visualiser le signal de SCMd avec la référence Sternum ou right Earlobe
emg_evoked.plot()

In [ ]:
bip105dB500Hz = epochs['500Hz105dB'].average()
mask = epochs['mask500Hz90dB'].average()
bip60db500Hz = epochs['500Hz60dB'].average()
bip105dB2500Hz = epochs['2500Hz105dB'].average()

# cVEMPs

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt

# Affichage des figures
evokeds = dict(bip105dB500Hz=bip105dB500Hz,mask=mask,bip105dB2500Hz=bip105dB2500Hz,bip60db500Hz=bip60db500Hz)
picks = ['SCMd']
colors = dict(mask="Purple",bip105dB500Hz="CornFlowerBlue",bip60db500Hz="Crimson",bip105dB2500Hz="green")
fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(evokeds, picks=picks, ylim=dict(eeg=[-10, 10]), invert_y=True, colors=colors, axes=ax)

# Ajout des lignes pointillées bleues à 0.013 et 0.023
p13 = ax.axvline(0.013, color='blue', linestyle='--')
p23 = ax.axvline(0.023, color='blue', linestyle='--')

# Texte pour P13 et N23
ax.text(0.010, -105, 'P13', color='blue', fontsize=14)
ax.text(0.020, -105, 'N23', color='blue', fontsize=14)

# Augmentation de la taille de la police des axes
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Modification de la taille de la police pour les labels des axes
plt.xlabel('time (s)', fontsize=16)
plt.ylabel('µV', fontsize=16)

# Affichage des figures
plt.show()

In [ ]:
# sauvegarde les figures des réponses moyennées pour chaque type de son
bip105dB500Hz.save('D:/Stage_M2/YBassiscVEMPs_bip105dB500Hz-ave.fif')
bip105dB2500Hz.save('D:/Stage_M2/YBassiscVEMPs_bip105dB2500Hz-ave.fif')
bip60db500Hz.save('D:/Stage_M2/YBassiscVEMPs_bip60db500Hz-ave.fif')
mask.save('D:/Stage_M2/YBassiscVEMPs_mask-ave.fif')

# Save oVEMPs

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt

# Affichage des figures
evokeds = dict(bip105dB500Hz=bip105dB500Hz,mask=mask,bip105dB2500Hz=bip105dB2500Hz,bip60db500Hz=bip60db500Hz)
picks = ['IOg']
colors = dict(mask="Purple",bip105dB500Hz="CornFlowerBlue",bip60db500Hz="Crimson",bip105dB2500Hz="green")
fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(evokeds, picks=picks, ylim=dict(eeg=[-10, 10]), invert_y=True, colors=colors, axes=ax)

# Ajout des lignes pointillées bleues à 0.013 et 0.023
p13 = ax.axvline(0.013, color='blue', linestyle='--')
p23 = ax.axvline(0.023, color='blue', linestyle='--')

# Texte pour P13 et N23
ax.text(0.010, -105, 'P13', color='blue', fontsize=14)
ax.text(0.020, -105, 'N23', color='blue', fontsize=14)

# Augmentation de la taille de la police des axes
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Modification de la taille de la police pour les labels des axes
plt.xlabel('time (s)', fontsize=16)
plt.ylabel('µV', fontsize=16)

# Affichage des figures
plt.show()

In [ ]:
# sauvegarde les figures des réponses moyennées pour chaque type de son
bip105dB500Hz.save('D:/Stage_M2_Paul/JK/JKoVEMPs_bip105dB500Hz-ave.fif')
bip105dB2500Hz.save('D:/Stage_M2_Paul/JK/JKoVEMPs_bip105dB2500Hz-ave.fif')
bip60db500Hz.save('D:/Stage_M2_Paul/JK/JKoVEMPs_bip60db500Hz-ave.fif')
mask.save('D:/Stage_M2_Paul/JK/JKoVEMPs_mask-ave.fif')

# GRAND AVERAGE cVEMP

In [25]:
import os

subs = ['MC', 'SB', 'MS', 'AA', 'LB', 'YB', 'EN', 'CD', 'CL', 'JK', 'LS', 'RM', 'AV', 'CA', 'AL', 'LP', 'MV', 'PK']
sounds = ['bip60db500Hz', 'bip105db500Hz', 'bip105db2500Hz', 'mask']

all_evokeds_assis_bip60db500Hz = []
all_evokeds_assis_bip105db500Hz = []
all_evokeds_assis_bip105db2500Hz = []
all_evokeds_assis_mask = []

for sub in subs:
    print(sub)
    
    for s in sounds:
        print(s)
        
        if s == 'bip60db500Hz':
            sound_ok = '500Hz60dB'
        elif s == 'bip105db500Hz':
            sound_ok = '500Hz105dB'
        elif s == 'bip105db2500Hz':
            sound_ok = '2500Hz105dB'
        else:
            sound_ok = 'mask500Hz90dB'
            
        file_path = os.path.join('D:/Stage_M2/participants', sub, sub + 'cVEMPs_' + s + '-ave.fif')
        
        evoked = mne.read_evokeds(file_path, condition=sound_ok)
        
        if s == 'bip60db500Hz':
            all_evokeds_assis_bip60db500Hz.append(evoked)
        elif s == 'bip105db500Hz':
            all_evokeds_assis_bip105db500Hz.append(evoked)
        elif s == 'bip105db2500Hz':
            all_evokeds_assis_bip105db2500Hz.append(evoked)
        else:
            all_evokeds_assis_mask.append(evoked)
            
    print()
    print()

print(len(all_evokeds_assis_bip60db500Hz))
print(len(all_evokeds_assis_bip105db500Hz))
print(len(all_evokeds_assis_bip105db2500Hz))
print(len(all_evokeds_assis_mask))


MC
bip60db500Hz
Reading D:\Stage_M2\participants\MC\MCcVEMPs_bip60db500Hz-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =     -30.27 ...     200.20 ms (500Hz60dB)
        0 CTF compensation matrices available
        nave = 200 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.03, 0] sec)
bip105db500Hz
Reading D:\Stage_M2\participants\MC\MCcVEMPs_bip105db500Hz-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =     -30.27 ...     200.20 ms (500Hz105dB)
        0 CTF compensation matrices available
        nave = 198 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.03, 0] sec)
bip105db2500Hz
Reading D:\Stage_M2\participants\MC\MCcVEMPs_bip105db2500Hz-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =  

        0 CTF compensation matrices available
        nave = 200 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.03, 0] sec)


YB
bip60db500Hz
Reading D:\Stage_M2\participants\YB\YBcVEMPs_bip60db500Hz-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =     -30.27 ...     200.20 ms (500Hz60dB)
        0 CTF compensation matrices available
        nave = 200 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.03, 0] sec)
bip105db500Hz
Reading D:\Stage_M2\participants\YB\YBcVEMPs_bip105db500Hz-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =     -30.27 ...     200.20 ms (500Hz105dB)
        0 CTF compensation matrices available
        nave = 199 - aspect type = 100
No projector specified for this dataset. Please consider the me

Loaded Evoked data is baseline-corrected (baseline: [-0.03, 0] sec)
mask
Reading D:\Stage_M2\participants\JK\JKcVEMPs_mask-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =     -30.27 ...     200.20 ms (mask500Hz90dB)
        0 CTF compensation matrices available
        nave = 200 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.03, 0] sec)


LS
bip60db500Hz
Reading D:\Stage_M2\participants\LS\LScVEMPs_bip60db500Hz-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =     -30.27 ...     200.20 ms (500Hz60dB)
        0 CTF compensation matrices available
        nave = 200 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.03, 0] sec)
bip105db500Hz
Reading D:\Stage_M2\participants\LS\LScVEMPs_bip105db500Hz-ave.fif ...
Isotrak no

    Found the data of interest:
        t =     -30.27 ...     200.20 ms (2500Hz105dB)
        0 CTF compensation matrices available
        nave = 199 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.03, 0] sec)
mask
Reading D:\Stage_M2\participants\AL\ALcVEMPs_mask-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =     -30.27 ...     200.20 ms (mask500Hz90dB)
        0 CTF compensation matrices available
        nave = 200 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.03, 0] sec)


LP
bip60db500Hz
Reading D:\Stage_M2\participants\LP\LPcVEMPs_bip60db500Hz-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =     -30.27 ...     200.20 ms (500Hz60dB)
        0 CTF compensation matrices available
        nave = 200 - aspect ty

In [26]:
#grand averages cVEMPs pour tous types de sons 
grand_average_cVEMPs_bip105dB2500Hz = mne.grand_average(all_evokeds_assis_bip105db2500Hz, interpolate_bads=True, drop_bads=True)
grand_average_cVEMPs_bip105db500Hz = mne.grand_average(all_evokeds_assis_bip105db500Hz, interpolate_bads=True, drop_bads=True)
grand_average_cVEMPs_bip60db500Hz = mne.grand_average(all_evokeds_assis_bip60db500Hz, interpolate_bads=True, drop_bads=True)
grand_average_cVEMPs_mask = mne.grand_average(all_evokeds_assis_mask, interpolate_bads=True, drop_bads=True)

Identifying common channels ...
Dropped the following channels:
['IOd', 'IOg', 'Eard']
Identifying common channels ...
Dropped the following channels:
['IOd', 'IOg', 'Eard']
Identifying common channels ...
Dropped the following channels:
['IOd', 'IOg', 'Eard']
Identifying common channels ...
Dropped the following channels:
['IOd', 'IOg', 'Eard']


In [46]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.ticker as ticker
import mne
%matplotlib qt

evokeds_ga = dict()

#evokeds_ga['Activating_sound'] = grand_average_cVEMPs_bip105db500Hz
#evokeds_ga["Frequency_control_sound"] = grand_average_cVEMPs_bip105dB2500Hz
evokeds_ga["Intensity_control_sound"] = grand_average_cVEMPs_bip60db500Hz
#evokeds_ga['Masked_sound'] = grand_average_cVEMPs_mask

print(evokeds_ga)

colors = dict(Intensity_control_sound="Crimson")#Frequency_control_sound="green")#Activating_sound="CornFlowerBlue")#Masked_sound="Purple")
###,,,
              #   

picks = ['SCMd']
fig, ax = plt.subplots()

# Tracer la courbe sur l'axe principal
mne.viz.plot_compare_evokeds(evokeds_ga, picks=picks, invert_y=True, ylim=dict(eeg=[-40, 40]), colors=colors, show=False, axes=ax)

# Positionnement de la légende
legend = ax.legend(fontsize=14, loc='lower right', bbox_to_anchor=(1.0, 0.0))
legend.set_title(None)

# Agrandissement de la taille de la police de la légende
for text in legend.texts:
    text.set_fontsize(12)
# Ajout des lignes pointillées bleues à 0.013 et 0.023
p13 = plt.axvline(0.013, color='blue', linestyle='--')
p23 = plt.axvline(0.023, color='blue', linestyle='--')

# Texte pour P13 et N23
#plt.text(0.010, -40, 'P13', color='blue', fontsize=14)
#plt.text(0.020, -40, 'N23', color='blue', fontsize=14)
# Aggrandissement de la taille de la police et mise en gras des titres
font_title = fm.FontProperties(weight='bold', size=16)
ax.set_xlabel('time (ms)', fontproperties=font_title)
ax.set_ylabel('µV', fontproperties=font_title)

# Agrandissement de la taille de la police des étiquettes d'axe
font_ticklabels = fm.FontProperties(size=14)
ax.tick_params(axis='both', labelsize=14)
plt.xticks(fontproperties=font_ticklabels)
plt.yticks(fontproperties=font_ticklabels)

# Formatage des valeurs de l'axe x en millisecondes
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x * 1000:.0f}'))
ax.set_xticklabels([f'{x * 1000:.0f}' for x in ax.get_xticks()])

# Affichage de la figure
plt.show()

{'Intensity_control_sound': <Evoked | 'Grand average (n = 18)' (average, N=18), -0.030273 – 0.2002 sec, baseline -0.03 – 0 sec, 66 ch, ~193 kB>}


C:\Users\kobli\AppData\Local\Temp\ipykernel_6016\689169163.py:24: RuntimeWarning: Cannot find channel coordinates in the supplied Evokeds. Not showing channel locations.
  mne.viz.plot_compare_evokeds(evokeds_ga, picks=picks, invert_y=True, ylim=dict(eeg=[-40, 40]), colors=colors, show=False, axes=ax)


# GRAND AVERAGE oVEMP

In [2]:
import mne
subs = ['MC','SB','MS','AA','LB','YB','EN','CD','CL','JK','LS','RM','AV','CA','AL','LP','MV','PK']
sounds = ['bip60db500Hz','bip105db500Hz','bip105db2500Hz','mask']

all_evokeds_oVEMPs_bip60db500Hz = []
all_evokeds_oVEMPs_bip105db500Hz = []
all_evokeds_oVEMPs_bip105db2500Hz = []
all_evokeds_oVEMPs_mask = []


for sub in subs:
    print(sub)
    for s in sounds:
        print(s)
        if s == 'bip60db500Hz':
            sound_ok = '500Hz60dB'
            evoked = mne.read_evokeds('D:/Stage_M2/'+ sub + 'oVEMPs' + '_' + s + '-ave.fif',
                                 condition = sound_ok )
            all_evokeds_oVEMPs_bip60db500Hz.append(evoked)
        elif s == 'bip105db500Hz':
            sound_ok = '500Hz105dB'
            evoked = mne.read_evokeds('D:/Stage_M2/' + sub + 'oVEMPs' + '_' + s + '-ave.fif',
                                 condition = sound_ok )
            all_evokeds_oVEMPs_bip105db500Hz.append(evoked)
        elif s == 'bip105db2500Hz':
            sound_ok = '2500Hz105dB'
            evoked = mne.read_evokeds('D:/Stage_M2/' + sub + 'oVEMPs' + '_' + s + '-ave.fif',
                                 condition = sound_ok )
            all_evokeds_oVEMPs_bip105db2500Hz.append(evoked)
        else:
            sound_ok = 'mask500Hz90dB'
            evoked = mne.read_evokeds('D:/Stage_M2/'+ sub + 'oVEMPs' + '_' + s + '-ave.fif',
                                 condition = sound_ok )
            all_evokeds_oVEMPs_mask.append(evoked)
    print()
    print()
                    
    
print(len(all_evokeds_oVEMPs_bip60db500Hz))
print(len(all_evokeds_oVEMPs_bip105db500Hz))
print(len(all_evokeds_oVEMPs_bip105db2500Hz))
print(len(all_evokeds_oVEMPs_mask))

MC
bip60db500Hz
Reading D:\Stage_M2\MCoVEMPs_bip60db500Hz-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =     -49.80 ...     299.80 ms (500Hz60dB)
        0 CTF compensation matrices available
        nave = 599 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.05, 0] sec)
bip105db500Hz
Reading D:\Stage_M2\MCoVEMPs_bip105db500Hz-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =     -49.80 ...     299.80 ms (500Hz105dB)
        0 CTF compensation matrices available
        nave = 599 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.05, 0] sec)
bip105db2500Hz
Reading D:\Stage_M2\MCoVEMPs_bip105db2500Hz-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =     -49.80 ...     299.80 ms (2500Hz105dB)
      

Isotrak not found
    Found the data of interest:
        t =     -49.80 ...     299.80 ms (500Hz60dB)
        0 CTF compensation matrices available
        nave = 599 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.05, 0] sec)
bip105db500Hz
Reading D:\Stage_M2\YBoVEMPs_bip105db500Hz-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =     -49.80 ...     299.80 ms (500Hz105dB)
        0 CTF compensation matrices available
        nave = 599 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.05, 0] sec)
bip105db2500Hz
Reading D:\Stage_M2\YBoVEMPs_bip105db2500Hz-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =     -49.80 ...     299.80 ms (2500Hz105dB)
        0 CTF compensation matrices available
        nave = 598 - aspect ty

        0 CTF compensation matrices available
        nave = 599 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.05, 0] sec)
bip105db500Hz
Reading D:\Stage_M2\LSoVEMPs_bip105db500Hz-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =     -49.80 ...     299.80 ms (500Hz105dB)
        0 CTF compensation matrices available
        nave = 599 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.05, 0] sec)
bip105db2500Hz
Reading D:\Stage_M2\LSoVEMPs_bip105db2500Hz-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =     -49.80 ...     299.80 ms (2500Hz105dB)
        0 CTF compensation matrices available
        nave = 597 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evok

No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.05, 0] sec)
bip105db500Hz
Reading D:\Stage_M2\LPoVEMPs_bip105db500Hz-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =     -49.80 ...     299.80 ms (500Hz105dB)
        0 CTF compensation matrices available
        nave = 599 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.05, 0] sec)
bip105db2500Hz
Reading D:\Stage_M2\LPoVEMPs_bip105db2500Hz-ave.fif ...
Isotrak not found
    Found the data of interest:
        t =     -49.80 ...     299.80 ms (2500Hz105dB)
        0 CTF compensation matrices available
        nave = 596 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.05, 0] sec)
mask
Reading D:\Stage_M2\LPo

In [3]:
%matplotlib qt

In [4]:
grand_average_oVEMPs_bip105dB2500Hz = mne.grand_average(all_evokeds_oVEMPs_bip105db2500Hz, interpolate_bads=True, drop_bads=True)
grand_average_oVEMPs_bip105db500Hz = mne.grand_average(all_evokeds_oVEMPs_bip105db500Hz, interpolate_bads=True, drop_bads=True)
grand_average_oVEMPs_bip60db500Hz = mne.grand_average(all_evokeds_oVEMPs_bip60db500Hz, interpolate_bads=True, drop_bads=True)
grand_average_oVEMPs_mask = mne.grand_average(all_evokeds_oVEMPs_mask, interpolate_bads=True, drop_bads=True)

Identifying common channels ...
Dropped the following channels:
['SCMd', 'Sternum']
Identifying common channels ...
Dropped the following channels:
['SCMd', 'Sternum']
Identifying common channels ...
Dropped the following channels:
['SCMd', 'Sternum']
Identifying common channels ...
Dropped the following channels:
['SCMd', 'Sternum']


In [17]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.ticker as ticker
import mne
%matplotlib qt

evokeds_ga = dict()

#evokeds_ga['Activating_sound'] = grand_average_oVEMPs_bip105db500Hz
#evokeds_ga["Frequency_control_sound"] = grand_average_oVEMPs_bip105dB2500Hz
#evokeds_ga["Intensity_control_sound"] = grand_average_oVEMPs_bip60db500Hz
evokeds_ga['Masked_sound'] = grand_average_oVEMPs_mask

print(evokeds_ga)

colors = dict(Masked_sound="Purple")   #Frequency_control_sound="green")#Activating_sound="CornFlowerBlue")#Intensity_control_sound="Crimson")#,,,
              #
picks = ['IOg']
fig, ax = plt.subplots()

# Tracer la courbe sur l'axe principal
mne.viz.plot_compare_evokeds(evokeds_ga, picks=picks, invert_y=True, ylim=dict(eeg=[-2, 2]), colors=colors, show=False, axes=ax)

# Aggrandissement de la taille de la police
font = fm.FontProperties(weight='bold', size=12)
ax.tick_params(axis='both', labelsize=12)
plt.xticks(fontproperties=font)
plt.yticks(fontproperties=font)
# Ajout des lignes pointillées bleues à 0.013 et 0.023
p13 = plt.axvline(0.010, color='blue', linestyle='--')
p23 = plt.axvline(0.0222, color='blue', linestyle='--')

# Texte pour P13 et N23
plt.text(0.008, -2, 'P10', color='blue', fontsize=14)
plt.text(0.018, -2, 'N22', color='blue', fontsize=14)
# Formatage des valeurs de l'axe x en millisecondes
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x * 1000:.0f}'))
ax.set_xticklabels([f'{x * 1000:.0f}' for x in ax.get_xticks()])

# Modification de la taille de la police pour les labels des axes
plt.xlabel('time (ms)', fontproperties=font)
plt.ylabel('µV', fontproperties=font)

# Agrandissement de la légende
legend = ax.legend(fontsize=14)
legend.set_title(None)

# Agrandissement de la taille de la police de la légende
for text in legend.texts:
    text.set_fontsize(14)
    #text.set_fontweight('bold')

# Affichage de la figure
plt.show()

NameError: name 'grand_average_assis_bip105db500Hz' is not defined

# Evoked par sujets

In [ ]:
subs = ['MC','SB','MS','AA','LB','YB','EN','CD','CL']
sounds = ['bip60db500Hz','bip105db500Hz','bip105db2500Hz','mask']

all_evokeds_cVEMPs_bip60db500Hz = []
all_evokeds_cVEMPs_bip105db500Hz = []
all_evokeds_cVEMPs_bip105db2500Hz = []
all_evokeds_cVEMPs_mask = []

data_dfs = []

for sub in subs:
    print(sub)
    for s in sounds:
        print(s)
        if s == 'bip60db500Hz':
            sound_ok = '500Hz60dB'
            evoked = mne.read_evokeds('D:/Stage_M2_Paul/' + sub + '/'+ sub + 'cVEMPs' + '_' + s + '-ave.fif',
                                 condition = sound_ok )
            all_evokeds_cVEMPs_bip60db500Hz.append(evoked)
            evoked_df = evoked.to_data_frame()
            evoked_df['subject'] = sub
            evoked_df['sound'] = s
        elif s == 'bip105db500Hz':
            sound_ok = '500Hz105dB'
            evoked = mne.read_evokeds('D:/Stage_M2_Paul/' + sub + '/'+ sub + 'cVEMPs' + '_' + s + '-ave.fif',
                                 condition = sound_ok )
            all_evokeds_cVEMPs_bip105db500Hz.append(evoked)
            evoked_df = evoked.to_data_frame()
            evoked_df['subject'] = sub
            evoked_df['sound'] = s
        elif s == 'bip105db2500Hz':
            sound_ok = '2500Hz105dB'
            evoked = mne.read_evokeds('D:/Stage_M2_Paul/' + sub + '/'+ sub + 'cVEMPs' + '_' + s + '-ave.fif',
                                 condition = sound_ok )
            all_evokeds_cVEMPs_bip105db2500Hz.append(evoked)
            evoked_df = evoked.to_data_frame()
            evoked_df['subject'] = sub
            evoked_df['sound'] = s
        else:
            sound_ok = 'mask500Hz90dB'
            evoked = mne.read_evokeds('D:/Stage_M2_Paul/' + sub + '/'+ sub + 'cVEMPs' + '_' + s + '-ave.fif',
                                 condition = sound_ok )
            all_evokeds_cVEMPs_mask.append(evoked)
            evoked_df = evoked.to_data_frame()
            evoked_df['subject'] = sub
            evoked_df['sound'] = s
            
        data_dfs.append(evoked_df)    
    
    print()
    print()
                    
    
print(len(all_evokeds_cVEMPs_bip60db500Hz))
print(len(all_evokeds_cVEMPs_bip105db500Hz))
print(len(all_evokeds_cVEMPs_bip105db2500Hz))
print(len(all_evokeds_cVEMPs_mask))

In [ ]:
import pandas as pd
df_all_mean_traces = pd.concat(data_dfs)
df_all_mean_traces

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib qt

In [ ]:
g = sns.relplot(df_all_mean_traces, kind='line',
            x='time',y='SCMd',hue='subject',
            row='sound',col='condition')

for ax in g.axes_dict.values():
    ax.axvline(0, ymin=0, ymax=1, color="0.1", dashes=(2, 1), zorder=0)

g.set_axis_labels("Time (s)", "Amplitude (µV)")
plt.gca().invert_yaxis()
plt.show()

In [ ]:
g = sns.relplot(df_all_mean_traces[((df_all_mean_traces['sound']=='bip60db500Hz')|
                                    (df_all_mean_traces['sound']=='mask'))], kind='line', 
                                   x='time',y='SCMd',hue='sound',col='subject', col_wrap=3)

for ax in g.axes_dict.values():
    ax.axvline(0, ymin=0, ymax=1, color="0.1", dashes=(2, 1), zorder=0)

g.set_axis_labels("Time (s)", "Amplitude (µV)")
#g.set_titles('FCz electrode')

plt.suptitle('SCMd', y = 1)
plt.gca().invert_yaxis()
plt.show()